In [106]:
import json, csv, copy, re
from collections import defaultdict

In [107]:
with open('template_riah_save.json') as f_in:
    riah = json.load(f_in)

In [108]:
riah.keys()

dict_keys(['@type', 'sourceDb', 'cdmDb', 'tableToTableMaps', 'tableMapToFieldToFieldMaps'])

In [109]:
del riah['tableToTableMaps']

In [110]:
del riah['tableMapToFieldToFieldMaps']

In [111]:
del riah['cdmDb']

In [112]:
riah['sourceDb']['tables']['@items'][0].keys()

dict_keys(['@id', '@type', 'db', 'name', 'description', 'rowCount', 'rowsCheckedCount', 'comment', 'fields', 'isStem'])

In [113]:
table = {  
          "@id":101,
          "@type":"org.ohdsi.rabbitInAHat.dataModel.Table",
          "db":None,
          "name": "baseline",
          "rowCount": 99999,
          "comment":"",
          "isStem":False,
          "fields":{            
              "@type":"java.util.ArrayList",
              "@items":[]
          }
}

In [114]:
riah['sourceDb']['tables']['@items'][0]['fields']['@items'][0].keys()

dict_keys(['@id', '@type', 'table', 'name', 'comment', 'valueCounts', 'isNullable', 'type', 'description', 'maxLength', 'isStem', 'conceptIdHints', 'fractionEmpty', 'uniqueCount', 'fractionUnique'])

In [115]:
field = {
                "@type":"org.ohdsi.rabbitInAHat.dataModel.Field",
                "table":{"@ref":None},
                "name": None,
                "comment":"",
                "valueCounts":[],
                "isNullable":True,
                "type":None,
                "description":"",
                "fractionEmpty":None,
                "uniqueCount":None,
                "fractionUnique":None,
                "isStem":False,
                "conceptIdHints":[]
}

In [116]:
table_list = {}
i = 100
with open('prio_vars.csv') as f_in:
    rows = csv.DictReader(f_in)
    for row in rows:
        table_name = row['category_name']
        t = table_list.get(table_name)

        # By category_name
        if t == None:
            i += 1
            t = copy.deepcopy(table)
            t['name'] = table_name
            t['@id'] = i
            t['db'] = None
            t['comment'] = ""
            table_list[table_name] = t
        
        f = copy.deepcopy(field)
        field_notes, n_rep = re.subn(r'\s?</?\w+>\s?', ' ', row['notes'])
#         field_description = field_notes or row['field_name']
        f['table']['@ref'] = t['@id']
        f['name'] = f"{row['field_id']}-{int(row['n_visits'])-1}.{int(row['n_indices'])-1}"  # The maximum array index
        f['description'] = f"{field_notes[:400]}"  # otherwise text too big for description field
        f['type'] = row['type_name'] + ' ' + row['unit_name']
        f['uniqueCount'] = int(row['n_values']) if row['n_values'] else None
        f['comment'] = ""
        f['item_count'] = int(row['item_count']) if row['item_count'] else None
        
        # todo: add possible values (and value counts) for categoricals
        # todo: add full category path, to see what belongs to what (e.g. Summary Administration - Hospital inpatient - Health-related outcomes)
        # todo: in table description, add which field ids it concerns (implement with another loop through created tables)
        
        t['fields']['@items'].append(f)

In [117]:
riah['sourceDb']['tables']['@items'] = list(table_list.values())

In [118]:
with open('riah_baseline.json', 'w') as f_out:
    json.dump(riah, f_out)